In [1]:
import numpy as np
import pandas as pd

# 1. apply 函数

In [ ]:
"""
apply 函数可以接受一个自定义函数,可以将 DataFrame 的行列数据传递给自定义函数处理
apply 函数类似于一个for循环,便利行列元素,但比for循环效率更高
"""

## 1.1 apply 函数作用于 Series 对象

In [33]:
# 1. 创建 df 对象,给两列值
df = pd.DataFrame({'a':[10,20,30],'b':[20,30,40]})
df

,a,b
0,10,20
1,20,30
2,30,40


In [35]:
# 2. 演示 apply() 函数,操作 Series 对象
# 需求1: 自定义函数 my_func1()  实现接受 Series 对象,然后使其每个值变成 其平方结果
def my_func1(x):
    return x ** 2

# 传入 Series 对象,调用 my_func1 函数
# 细节 apply()函数会把 my_fun1 函数所用到 Series的每个对象上
df.a.apply(my_func1)

0    100
1    400
2    900
Name: a, dtype: int64

In [39]:
# 需求2: apply传入,需要多个参数的函数,例如: 自定义函数 my_func2(x,e),实现计算 x的e次方
def my_fun2(x,e):
    return x ** e

df.a.apply(my_fun2,e=3)

0     1000
1     8000
2    27000
Name: a, dtype: int64

## 1.2 apply 函数作用于 DataFrame 对象

In [64]:
# 细节 apply() 函数作用于 DF独享.默认传入的是 axis=0 整列数据, 不是Series一样,逐个元素传递
# 1. 把上述的 my_func1() 函数,作用到 DF 对象
df.apply(my_func1)

# 2. 自定义函数 my_func3() , 看看 df 对象到底传入的是什么
def my_func3(x):
    print(f'x的内容:{x}')
    print(f'x的类型:{type(x)}')

# 3. 调用上述的my_func3(),作用于:df对象
# df.apply(my_func3) # 默认传入的是整列 series对象

# 4.加入 axis 参数
# df.apply(my_func3,axis=1)       # 0:列 1:行

# 如下是一种错误示例,为了引出函数的向量化
def avg_3(x,y,z):
    return (x+y+z) / 3

def avg_3_mod(x):
    n1 = x[0]
    n2 = x[1]
    n3 = x[2]
    return (n1 + n2 + n3) / 3
df.apply(avg_3_mod)


a    20.0
b    30.0
dtype: float64

## 2. apply() 函数案例-泰坦尼克号数据集

In [66]:
"""
需求: 自定义函数,分别计算 泰坦尼克号的数据集 某列的缺失值个数,某列的缺失值占比,某列的非缺失值占比
# 1. 读取数据源 获取 df 对象
"""
titanic = pd.read_csv("data/titanic_train.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [70]:
# 2. 定义函数 实现各种需求
# 需求1: count_missing(vec),计算某列的缺失值个数
def count_missing(vec):
    # vec就是接收到 df 独享的某列或者某行数据
    return pd.isnull(vec).sum()

# 需求2: prop_missing(vec) 计算某列缺失值占比
def prop_missing(vec):
    # 缺失值占比公式: 某列缺失值数量 / 某列元素总个数
    return count_missing(vec) / vec.size

# 需求3: prop_complete(vec),计算某列的 非缺失值占比
def prop_complete(vec):
    return 1- prop_missing(vec)

titanic.apply(count_missing)
titanic.apply(prop_missing)
titanic.apply(prop_complete)

PassengerId    1.000000
Survived       1.000000
Pclass         1.000000
Name           1.000000
Sex            1.000000
Age            0.801347
SibSp          1.000000
Parch          1.000000
Ticket         1.000000
Fare           1.000000
Cabin          0.228956
Embarked       0.997755
dtype: float64

## 3.向量化函数对象介绍 ,np.vectorize

In [77]:
 # 1. 创建 df 对象,给两列值
df = pd.DataFrame({'a':[10,20,30],'b':[20,30,40]})
df

,a,b
0,10,20
1,20,30
2,30,40


In [80]:
def avg_2(x,y):
    return (x + y) / 2

avg_2(x=df['a'],y=df['b'])

# 改造上述代码,程序出问题
def avg_2_mod(x,y):
    if x == 20:  # 这里会出错,因为 x 是向量(简单理解为一堆纸),20 是标量(1个值)
        return np.nan
    else:
        return (x + y) / 2

avg_2_mod(df['a'],df['b'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [82]:
# 解决思路: 通过 np.vectorize 将上述的函数转成:向量化函数,如果函数中遇到向量了,则内部会自动遍历

@np.vectorize
def avg_2_mod(x,y):
    if x == 20:  # 这里会出错,因为 x 是向量(简单理解为一堆纸),20 是标量(1个值)
        return np.nan
    else:
        return (x + y) / 2

avg_2_mod(df['a'],df['b'])


array([15., nan, 35.])

In [83]:
## 方式二: lambda 方式
df.apply(lambda x: x ** 2)

,a,b
0,100,400
1,400,900
2,900,1600


# 分组聚合演变


In [84]:
# 需求4: 使用自定义函数,完成计算平均值
df2 = pd.DataFrame({
    '城市': ['北京', '北京', '上海', '上海', '广州', '广州'],
    '部门': ['销售', '销售', '技术', '技术', '销售', '技术'],
    '收入': [8000, 9000, 10000, 11000, 7000, 9500]
})

In [90]:
def my_mean(col):
    # return col.mean()
    return  col.sum() / col.size

# 传入我们自定义的函数,计算平均值
# df2.groupby(['部门','城市']).agg({'收入':my_mean})
df2.groupby(['部门','城市'])['收入'].agg(my_mean)





部门  城市
技术  上海    10500.0
    广州     9500.0
销售  北京     8500.0
    广州     7000.0
Name: 收入, dtype: float64

In [93]:
# 需求6 agg() 函数,同时传入多个函数
df2.groupby(['部门','城市'])['收入'].agg(['sum','mean','max'])

sum     mean    max
部门 城市                       
技术 上海  21000  10500.0  11000
   广州   9500   9500.0   9500
销售 北京  17000   8500.0   9000
   广州   7000   7000.0   7000

In [97]:
# 窗口函数展示
df2['部门平均'] = df2.groupby('部门')['收入'].transform("mean")
df2

,城市,部门,收入,部门平均
0,北京,销售,8000,8000.000000
1,北京,销售,9000,8000.000000
2,上海,技术,10000,10166.666667
3,上海,技术,11000,10166.666667
4,广州,销售,7000,8000.000000
5,广州,技术,9500,10166.666667


## 分组过滤

In [104]:
tips = pd.read_csv("data/tips.csv")
# tips

# 查看吃饭人数的分布情况
tips['size'].value_counts()

# 发现 1 5 6 个人吃饭次数较少,直接过滤掉这些数据
tmp_df = tips.groupby("size").filter(lambda x : x['size'].count() > 10)

tmp_df['size'].value_counts()


size
2    156
3     38
4     37
Name: count, dtype: int64

# 数据透视表

In [ ]:
"""
透视表是一种交互式的表,可以进行某些计算,如求和计数等. 所进行的计算与数据跟数据透视表中的排序有关系
透视数据表可以动态改变他们版面的位置,一百年按照不同方式分析数据,也可以重新安排行号、列标和页字段。每一次改变版面布置时,数据透视表会立即按照新的布局重新计算你数据.另外,如果原始数据发生改变则可以更新数据透视表
在 Excel 数据分时时,透视表是很常见的功能, pandas 也提供了透视表功能,对应 的 API 为 pivot_table
"""

"""
pivot_table函数介绍
pandas 有两个 pivot_table 函数
pandas.pivot_table
pandas.DataFrame.pivot_table
pandas.pivot_table 比 pandas.DataFrame.pivot_table 多一个Data参数,data 就是一个 dataFrame ,实际上这两个函数相同
pivot_tabe 参数中最重要的四个参数 values index columns aggfunc 

"""